# 017-ORF

担当: 久野 朗広

## 転写はどこからでも始まる可能性がある

### 図1：特定のORF（開始コドンと停止コドンが示された）の模式図

「DNAからRNAへの転写」では、DNAがRNAに転写されるプロセスについて説明しました。また、「RNAからタンパク質への翻訳」では、RNAがアミノ酸の鎖に翻訳されてタンパク質が作られる過程を見てきました。これら2つのプロセスを一連のステップと捉え、DNA文字列を直接タンパク質文字列へと変換することで、DNAコドン表の使用が求められます。

しかし、DNAから直接タンパク質への変換を試みる際に、すぐに3つの複雑さが立ちはだかります。

1. **すべてのDNAがRNAに転写されるわけではない**  
   いわゆる「ジャンクDNA」は、細胞の機能にとって実用的な目的を持たないように見えます。

2. **RNA上の任意の位置から翻訳を開始できる**  
   つまり、DNA文字列の任意の部分文字列が、開始コドンで始まり停止コドンで終わり、途中に他の停止コドンがなければ、翻訳のテンプレートとして機能します（図1参照）。  
   この結果、同じRNA文字列でも、コドン（3文字）をどう区切るかによって3通りの翻訳が可能になります。  
   たとえば、`...AUGCUGAC...` は次のように3通りに翻訳できます：
   - `...AUG CUG...`
   - `...UGC UGA...`
   - `...GCU GAC...`  
   通常、これらはまったく異なるタンパク質文字列を生じさせます。

---

### 問題

DNA二重らせんのどちらの鎖も、RNA転写におけるコード鎖となる可能性があります。したがって、1つのDNA文字列から合計6通りの読み枠（リーディングフレーム）が導かれます：

- 3通りはDNA文字列自身を読み取ることで得られます。
- 残りの3通りは、その逆相補鎖を読み取ることで得られます。

**オープンリーディングフレーム（ORF）**とは、開始コドンから始まり、途中に他の停止コドンを挟まずに停止コドンで終わる領域のことです。したがって、候補となるタンパク質文字列は、ORFをアミノ酸に翻訳することで、停止コドンに達するまでに得られたものになります。

---

### 入力：  
FASTA形式で与えられた長さ最大1 kbpのDNA文字列 `s`

### 出力：  
`s` のORFから翻訳されたすべての**異なる候補タンパク質文字列**を返してください。文字列の出力順は任意です。

---

### サンプル入力

```
>Rosalind_99  
AGCCATGTAGCTAACTCAGGTTACATGGGGATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAATAAGCCTGAATGATCCGAGTAGCATCTCAG
```

### サンプル出力

```
MLLGSFRLIPKETLIQVAGSSPCNLS  
M  
MGMTPRLGLESLLE  
MTPRLGLESLLE
```


## 解答例

In [2]:
# コドン表（ストップコドンは *）
codon_table = {
    'ATA': 'I', 'ATC': 'I', 'ATT': 'I', 'ATG': 'M',
    'ACA': 'T', 'ACC': 'T', 'ACG': 'T', 'ACT': 'T',
    'AAC': 'N', 'AAT': 'N', 'AAA': 'K', 'AAG': 'K',
    'AGC': 'S', 'AGT': 'S', 'AGA': 'R', 'AGG': 'R',
    'CTA': 'L', 'CTC': 'L', 'CTG': 'L', 'CTT': 'L',
    'CCA': 'P', 'CCC': 'P', 'CCG': 'P', 'CCT': 'P',
    'CAC': 'H', 'CAT': 'H', 'CAA': 'Q', 'CAG': 'Q',
    'CGA': 'R', 'CGC': 'R', 'CGG': 'R', 'CGT': 'R',
    'GTA': 'V', 'GTC': 'V', 'GTG': 'V', 'GTT': 'V',
    'GCA': 'A', 'GCC': 'A', 'GCG': 'A', 'GCT': 'A',
    'GAC': 'D', 'GAT': 'D', 'GAA': 'E', 'GAG': 'E',
    'TCA': 'S', 'TCC': 'S', 'TCG': 'S', 'TCT': 'S',
    'TTC': 'F', 'TTT': 'F', 'TTA': 'L', 'TTG': 'L',
    'TAC': 'Y', 'TAT': 'Y', 'TAA': '*', 'TAG': '*',
    'TGC': 'C', 'TGT': 'C', 'TGA': '*', 'TGG': 'W'
}


## reモジュールを使う

In [46]:
import re

# コドン表（DNAベース、T表記）
codon_table = {
    'TTT': 'F', 'TTC': 'F', 'TTA': 'L', 'TTG': 'L',
    'CTT': 'L', 'CTC': 'L', 'CTA': 'L', 'CTG': 'L',
    'ATT': 'I', 'ATC': 'I', 'ATA': 'I', 'ATG': 'M',
    'GTT': 'V', 'GTC': 'V', 'GTA': 'V', 'GTG': 'V',
    'TCT': 'S', 'TCC': 'S', 'TCA': 'S', 'TCG': 'S',
    'CCT': 'P', 'CCC': 'P', 'CCA': 'P', 'CCG': 'P',
    'ACT': 'T', 'ACC': 'T', 'ACA': 'T', 'ACG': 'T',
    'GCT': 'A', 'GCC': 'A', 'GCA': 'A', 'GCG': 'A',
    'TAT': 'Y', 'TAC': 'Y', 'TAA': '*', 'TAG': '*',
    'CAT': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q',
    'AAT': 'N', 'AAC': 'N', 'AAA': 'K', 'AAG': 'K',
    'GAT': 'D', 'GAC': 'D', 'GAA': 'E', 'GAG': 'E',
    'TGT': 'C', 'TGC': 'C', 'TGA': '*', 'TGG': 'W',
    'CGT': 'R', 'CGC': 'R', 'CGA': 'R', 'CGG': 'R',
    'AGT': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R',
    'GGT': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G'
}

def revcomp(S):
    """DNA文字列の逆補鎖を返す関数"""
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    return ''.join(complement[base] for base in S[::-1])

def translate_orf(dna, regex_pattern):
    protein_set = set()
    for frame in range(3):
        sub = dna[frame:]
        for match in regex_pattern.finditer(sub):
            codon_seq = match.group(1)
            amino_acids = []
            for i in range(0, len(codon_seq), 3):
                codon = codon_seq[i:i+3]
                aa = codon_table.get(codon, '')
                if aa == '*' or len(codon) < 3:
                    break
                amino_acids.append(aa)
            protein_set.add(''.join(amino_acids))
    return protein_set

def parse_fasta(fasta):
    lines = fasta.strip().split('\n')
    return ''.join(line for line in lines if not line.startswith('>'))

In [47]:
# 入力FASTA
fasta_input = """>Rosalind_99  
AGCCATGTAGCTAACTCAGGTTACATGGGGATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAATAAGCCTGAATGATCCGAGTAGCATCTCAG"""

dna_seq = parse_fasta(fasta_input)
revcomp_seq = revcomp(dna_seq)

proteins = set()

# 肯定先読み ＆ 正規表現をコンパイルして高速化
regex_pattern = re.compile(r'(?=(ATG(?:...)*?(?:TAA|TAG|TGA)))')

for seq in [dna_seq, revcomp_seq]:
    proteins.update(translate_orf(seq, regex_pattern))

for p in proteins:
    print(p)


M
MLLGSFRLIPKETLIQVAGSSPCNLS
MTPRLGLESLLE
MGMTPRLGLESLLE
